In [1]:
### 1. Data Loaders + Imports ###
import torchvision
import torch.nn as nn
import torch.nn.functional as nnf
import torch
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, normalize
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

food_train = torchvision.datasets.Food101("./data", split="train")
food_test = torchvision.datasets.Food101("./data", split="test")

def collate_fn(examples):

  processed_images = []
  processed_labels = []

  for example in examples:

    tensor_image = to_tensor(example[0])
    # In linia de mai jos imaginea este normalizata astfel incat sa aiba toate valorile in
    # [-1, 1] in loc de [0, 255]
    normalized_tensor_image = normalize(tensor_image, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    vector_image = normalized_tensor_image.unsqueeze(0)
    processed_images.append(vector_image)

    label = np.array(example[1])
    tensor_label = torch.tensor(label)
    tensor_label = tensor_label.unsqueeze(0)
    processed_labels.append(tensor_label)

  torch_images = torch.cat(processed_images, dim=0)
  torch_labels = torch.cat(processed_labels, dim=0)

  return torch_images, torch_labels

train_loader = DataLoader(food_train, batch_size=64, shuffle=True, num_workers=2, collate_fn=collate_fn)
test_loader = DataLoader(food_test, batch_size=8, shuffle=False, collate_fn=collate_fn)

# print(train_loader.__getitem__(0))
# for index, batch in enumerate(train_loader):
#     print('Dimensiunile primei imagini din primul batch extras: ', batch[0][0].size())
#     break

C:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


cuda


In [2]:
import torch
from torchvision import transforms
from torchvision.datasets import Food101
from torch.utils.data import DataLoader

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images
])

# Specify the root directory where the dataset will be stored
root_dir = './data'

# Load the Food101 dataset
food101_dataset = Food101(root=root_dir, split='train', transform=transform, download=True)

# Create a DataLoader for the dataset
batch_size = 32
data_loader = DataLoader(food101_dataset, batch_size=batch_size, shuffle=True)

# You can now iterate over the data loader to get batches of data
for images, labels in data_loader:
    print(images.size(), labels.size())
    break


torch.Size([32, 3, 224, 224]) torch.Size([32])


In [3]:
### 2. Train & Test Accuracy Functions ###

def test_acc(net: nn.Module, test_loader: DataLoader):
  net.eval()

  total = 0
  correct = 0

  for test_images, test_labels in test_loader:
    test_images, test_labels = test_images.to(device), test_labels.to(device)
    total += len(test_images)
    out_class = torch.argmax(net(test_images))
    correct += torch.sum(out_class == test_labels)

  return correct / total * 100

def train_fn(epochs: int, train_loader: DataLoader, test_loader: DataLoader,
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  net.to(device)
  # Iteram prin numarul de epoci
  for e in range(epochs):
    net.train()
    # Iteram prin fiecare batch din dataloader
    for images, labels in train_loader:
      images, labels = images.to(device), labels.to(device)
      # Aplicam reteaua neurala pe imaginile din batch-ul curent
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe etichetele imaginilor
      loss = loss_fn(out, labels)
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a actualiza parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()

    print("Loss-ul la finalul epocii {} are valoarea {}".format(e + 1, loss.item()))

    # Calculam acuratetea
    acc = test_acc(net, test_loader)
    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e + 1, acc))

**Food101 using ResNet50 Pretrained + fine tuning**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torchvision.datasets import Food101
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import time

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the datasets
train_dataset = Food101(root='./data', split='train', transform=transform, download=True)
test_dataset = Food101(root='./data', split='test', transform=transform, download=True)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load pre-trained ResNet model
resnet = models.resnet50(pretrained=True)

# Freeze all layers except the final classification layer
for param in resnet.parameters():
    param.requires_grad = False

# Modify the final classification layer
num_classes = len(train_dataset.classes)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

start = time.time()

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')
resnet.to(device)
num_epochs = 10
for epoch in range(num_epochs):
    resnet.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

print(f'Training duration: {time.time()-start}')

# Evaluate the model on the test set
resnet.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet(images)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Test Accuracy: {accuracy:.4f}")

torch.save(resnet.state_dict(), 'resnet_model2_trying.pkl')


c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Device: cuda
Epoch [1/10], Loss: 2.4382
Epoch [2/10], Loss: 1.9899
Epoch [3/10], Loss: 1.8959
Epoch [4/10], Loss: 1.8194
Epoch [5/10], Loss: 1.7676
Epoch [6/10], Loss: 1.7222
Epoch [7/10], Loss: 1.6843
Epoch [8/10], Loss: 1.6441
Epoch [9/10], Loss: 1.6083
Epoch [10/10], Loss: 1.5849
Training duration: 4028.9360151290894
Test Accuracy: 0.5890
